In [9]:
import requests

PORT = "9200"
URL = "http://localhost:" + PORT + "/"

def parse_analysis(string):
    split_input = [x.strip() for x in string.split("\t") if x != ""]
    
    if len(split_input) != 3:
        return ()
    
    base, form_explicit, _disamb = split_input
    form = form_explicit.split(":")[0]
    
    return (base, form)

def analyze(string):
    content = requests.post(URL, data=string.encode('utf-8')).text
    analyzed_words = [parse_analysis(analysis) for analysis in content.split("\n")[1::2] if analysis != ""]
    
    return [analysis for analysis in analyzed_words if analysis != ()]
    
def is_valid_unigram(unigram):
    word, analysis = unigram
    return analysis != "interp" and not analysis.startswith("num")


In [10]:
analyze("ala ma kota\n\n\n\n12     123.")

ConnectionError: HTTPConnectionPool(host='localhost', port=9200): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fc3081eed68>: Failed to establish a new connection: [Errno 111] Connection refused',))

In [11]:
FILE_LIST = 'files.p'
DATA_DIR = "/run/media/maciej/Nowy/data/json/"
CHOSEN_YEAR = "2011"

In [12]:
import regex as re

judgment_types = {
    "I":  "A?C.*",
    "II": "A?U.*",
    "III": "A?K.*",
    "IV": "G.*",
    "V": "A?P.*",
    "VI": "R.*",
    "VII": "W.*",
    "VIII": "Am.*",
}

def get_judgment_type(number):
    for key, value in judgment_types.items():
        if re.search(value, number):
            return key
    print("Unmatched {}".format(number))

In [14]:
import os, json, pickle
from collections import defaultdict

import regex
from tqdm import tqdm

word_pattern = "\p{Letter}+"


def load_data():
    total_judgments = []
    files = pickle.load(open(FILE_LIST, 'rb'))
    
    for file in tqdm(files):
        if file.startswith("judgment"):
            file_path = os.path.join(DATA_DIR, file)

            with open(file_path, 'r') as f:
                data = json.load(f)
                judgments = [
                    (x["textContent"], x["courtCases"][0]["caseNumber"]) for x in data["items"] 
                    if x["judgmentDate"].startswith(CHOSEN_YEAR) and x["courtType"] in ("COMMON", "SUPREME")
                ]
                
            total_judgments += judgments
    
    print("Found {} judgments from common and supreme courts".format(len(total_judgments)))
    
    judgments_with_explanations = []    
    for judgment, _case_num in total_judgments:        
        match = regex.search("uzasadnienie", judgment, regex.IGNORECASE)        
        if match:
            judgments_with_explanations.append((judgment[match.end():], _case_num))
            
    print("{} of them have explanation section".format(len(judgments_with_explanations)))
    
    analyzed_judgments = defaultdict(lambda: [])
    raw_judgments = defaultdict(lambda: [])
    
    for judgment, case_number in tqdm(judgments_with_explanations):
        
        judgment = regex.sub("<.*?>", "", judgment)
        judgment = regex.sub("-\n(\p{Letter}+)", r"\1", judgment)
        judgment = judgment.lower()

        judgment_type = get_judgment_type(case_number)

        raw_judgments[judgment_type].append(judgment)
        analyzed_judgments[judgment_type].append(analyze(judgment))
    
    return analyzed_judgments, raw_judgments

analyzed_judgments, raw_judgments = load_data()

  0%|          | 0/3215 [00:00<?, ?it/s]

Found 3791 judgments from common and supreme courts
3215 of them have explanation section


  2%|▏         | 58/3215 [02:24<2:10:53,  2.49s/it]

Unmatched II Ns 576/08


  4%|▍         | 138/3215 [05:19<1:58:49,  2.32s/it]

...

 98%|█████████▊| 3138/3215 [2:02:28<03:00,  2.34s/it]

Unmatched SNO 42/11


 98%|█████████▊| 3158/3215 [2:03:08<02:13,  2.34s/it]

Unmatched SDI 22/11


100%|██████████| 3215/3215 [2:06:11<00:00,  2.35s/it]


In [24]:
# import pickle
# pickle.dump(list(analyzed_judgments.items()), open("analyzed_judgments.p", "wb"))
# pickle.dump(list(raw_judgments.items()), open("raw_judgments.p", "wb"))


In [13]:
print([len(x) for x in raw_judgments.values()])
print([len(x) for x in analyzed_judgments.values()])

NameError: name 'raw_judgments' is not defined

In [16]:
import pickle
analyzed_judgments = pickle.load(open('analyzed_judgments.p', 'rb'))
raw_judgments = pickle.load(open('raw_judgments.p', 'rb'))

In [8]:
most_frequent = ['na',
 'do',
 'nie',
 'art',
 'że',
 'przez',
 'ust',
 'się',
 'dzień',
 'jest',
 'oraz',
 'ustawa',
 'od',
 'postępowanie',
 'nr',
 'sąd',
 'punkt',
 'za',
 'tym',
 'to']

In [17]:
len(analyzed_judgments)


8